# CS506 HW5: Predicting Customer Churn Using KNN
------------------------------------------------
Prediction of customer churn in the banking industry using the K-Nearest Neighbors (KNN) Algorithm.


In [29]:
import numpy as np
import pandas as pd

In [30]:
def standard_scaler(data):
    """Applies standard scaling to the input data."""
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    scaled_data = (data - mean) / std
    return scaled_data

def one_hot_encoder(df, column):
    unique_values = df[column].unique()
    for value in unique_values:
        df[column + '_' + str(value)] = (df[column] == value).astype(int)
    df = df.drop(columns=[column])
    return df

def impute_missing_values(df):
    # Imputes missing values using the mean strategy.
    for col in df.columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mean())
    return df

In [31]:
# Define the KNN Class from Scratch
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # Store training data
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def predict_proba(self, X):
        X = np.array(X)
        predictions = []

        for x_test in X:
            # Compute distances between x_test and all training samples one by one
            distances = self.compute_distance(self.X_train, x_test)

            # Get the indices of the k nearest neighbors
            nearest_neighbors_indices = np.argpartition(distances, self.k)[:self.k]

            # Retrieve the labels of the nearest neighbors
            nearest_labels = self.y_train[nearest_neighbors_indices]

            # Calculate the probability for the positive class
            positive_class_prob = np.sum(nearest_labels) / self.k

            # Append both class probabilities [P(negative), P(positive)]
            predictions.append([1 - positive_class_prob, positive_class_prob])

        return np.array(predictions)

    def compute_distance(self, X1, X2):
        X1 = X1.astype(np.float64)
        X2 = X2.astype(np.float64)
        X2 = X2.reshape(1, -1)

        if self.distance_metric == 'euclidean':
          # Vectorized Euclidean distance
          return np.sqrt(np.sum((X1 - X2)**2, axis=1))
        elif self.distance_metric == 'manhattan':
          # Vectorize Manhattan distance
          return np.sum(np.abs(X1 - X2), axis=1)
        else:
          raise ValueError("Unsupported distance metric")
    
    def predict(self, X):
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)

In [32]:
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv("train.csv")
    test_data = pd.read_csv("test.csv")

    # Separate features and labels
    X_train = train_data.drop(columns=['Exited', 'CustomerId', 'Surname'])
    y_train = train_data['Exited'].values
    X_test = test_data.drop(columns=['CustomerId', 'Surname'])

    for col in ['Geography']:
        X_train = one_hot_encoder(X_train, col)
        X_test = one_hot_encoder(X_test, col)

    # Encode categorical features: Gender and Geography before imputation
    X_train['Gender'] = X_train['Gender'].map({'Male': 1, 'Female': 0})
    X_test['Gender'] = X_test['Gender'].map({'Male': 1, 'Female': 0})

    # Handle missing values for both train and test sets (impute)
    X_train = impute_missing_values(X_train)
    X_test = impute_missing_values(X_test)

    # Select only numerical columns for scaling
    numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

    # Standard scale numerical columns
    X_train[numerical_cols] = standard_scaler(X_train[numerical_cols].values)
    X_test[numerical_cols] = standard_scaler(X_test[numerical_cols].values)

    X_train, X_test, y_train = X_train.values, X_test.values, y_train
    
    return X_train, X_test, y_train

In [33]:
def calculate_roc_auc_manual(y_true, y_scores):
    thresholds = np.unique(y_scores)
    thresholds = np.sort(thresholds)[::-1]
    tpr_list, fpr_list = [], []
    P = np.sum(y_true == 1)
    N = np.sum(y_true == 0)
    for threshold in thresholds:
        y_pred = (y_scores >= threshold).astype(int)
        TP = np.sum((y_true == 1) & (y_pred == 1))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        tpr_list.append(TP / P if P > 0 else 0)
        fpr_list.append(FP / N if N > 0 else 0)
    return np.trapz(tpr_list, fpr_list)

In [34]:
def cross_validate_with_metrics(X, y, k, distance_metric, n_splits=5):
    X = np.array(X)
    y = np.array(y)
    
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]

    fold_size = len(X) // n_splits
    auc_list = []

    for i in range(n_splits):
        # Splitting data
        X_val = X[i * fold_size: (i + 1) * fold_size]
        y_val = y[i * fold_size: (i + 1) * fold_size]
        X_train = np.concatenate((X[:i * fold_size], X[(i + 1) * fold_size:]), axis=0)
        y_train = np.concatenate((y[:i * fold_size], y[(i + 1) * fold_size:]), axis=0)

        # Train KNN
        knn = KNN(k=k, distance_metric=distance_metric)
        knn.fit(X_train, y_train)

        # Predict probabilities and convert to predicted labels
        y_pred_proba = knn.predict_proba(X_val)[:, 1]

        y_pred = (y_pred_proba >= 0.5).astype(int)  # Threshold at 0.5

        # Compute metrics
        auc = calculate_roc_auc_manual(y_val, y_pred_proba)

        # Append metrics to lists
        auc_list.append(auc)

    # Return average of the metrics across the folds
    return {
        "auc": np.mean(auc_list)
    }

In [35]:
# Load and preprocess data
X, X_test, y = preprocess_data('train.csv', 'test.csv')

# Hyperparameter tuning to find the best value of k
results = []
best_k = None
best_distance_metric = None
best_score = 0

# Store predictions for each distance metric
euclidean_predictions = []
manhattan_predictions = []

# Iterate over k values from 1 to 20
for k in range(1, 21):  
    # Calculate AUC for Euclidean distance
    auc_euclidean = cross_validate_with_metrics(X, y, k, 'euclidean')["auc"]

    # Calculate AUC for Manhattan distance
    auc_manhattan = cross_validate_with_metrics(X, y, k, 'manhattan')["auc"]

    # Initialize KNN model for Euclidean distance and fit it
    knn_euclidean = KNN(k=k, distance_metric='euclidean')
    knn_euclidean.fit(X, y)
    # Predict probabilities for the Exited class (1) for test data
    euclidean_pred = knn_euclidean.predict_proba(X_test)[:, 1]  

    # Initialize KNN model for Manhattan distance and fit it
    knn_manhattan = KNN(k=k, distance_metric='manhattan')
    knn_manhattan.fit(X, y)
    manhattan_pred = knn_manhattan.predict_proba(X_test)[:, 1]  

    # Store predictions
    euclidean_predictions.append(euclidean_pred)
    manhattan_predictions.append(manhattan_pred)

    # Compare and store the better AUC score for each k
    if auc_euclidean > auc_manhattan:
        results.append((k, 'euclidean', auc_euclidean))
    else:
        results.append((k, 'manhattan', auc_manhattan))

    # Update the best AUC score
    if auc_manhattan > best_score:
        best_score = auc_manhattan
        best_k = k
        best_distance_metric = 'manhattan'
    if auc_euclidean > best_score:
        best_score = auc_euclidean
        best_k = k
        best_distance_metric = 'euclidean'

# Print the results with the higher AUC score for each k
print("Higher AUC score for each k:")
for result in results:
    print(f'k: {result[0]}, Distance Metric: {result[1]}, AUC: {result[2]:.4f}')

print(f'Optimal k: {best_k}, Distance Metric: {best_distance_metric}, Best AUC: {best_score:.4f}')

# Now generate the final predictions by choosing the higher probability for each test example
final_predictions = np.maximum(euclidean_predictions[best_k-1], manhattan_predictions[best_k-1])

# Create and save the submission file in the correct format
test_data = pd.read_csv('test.csv')  # Load original test file for CustomerId
submission = pd.DataFrame({'id': test_data['id'], 'Exited': final_predictions})
submission.to_csv('submissions.csv', index=False)

print("Test predictions saved to 'submissions.csv'")


Higher AUC score for each k:
k: 1, Distance Metric: manhattan, AUC: 0.8854
k: 2, Distance Metric: euclidean, AUC: 0.8062
k: 3, Distance Metric: euclidean, AUC: 0.8393
k: 4, Distance Metric: euclidean, AUC: 0.8565
k: 5, Distance Metric: manhattan, AUC: 0.8854
k: 6, Distance Metric: euclidean, AUC: 0.8736
k: 7, Distance Metric: manhattan, AUC: 0.8854
k: 8, Distance Metric: manhattan, AUC: 0.8854
k: 9, Distance Metric: manhattan, AUC: 0.8854
k: 10, Distance Metric: euclidean, AUC: 0.8893
k: 11, Distance Metric: euclidean, AUC: 0.8913
k: 12, Distance Metric: manhattan, AUC: 0.8854
k: 13, Distance Metric: manhattan, AUC: 0.8854
k: 14, Distance Metric: manhattan, AUC: 0.8854
k: 15, Distance Metric: manhattan, AUC: 0.8854
k: 16, Distance Metric: manhattan, AUC: 0.8854
k: 17, Distance Metric: euclidean, AUC: 0.8966
k: 18, Distance Metric: manhattan, AUC: 0.8854
k: 19, Distance Metric: manhattan, AUC: 0.8854
k: 20, Distance Metric: manhattan, AUC: 0.8854
Optimal k: 20, Distance Metric: manhatta